In [1]:
import glob
from bioutilities import Genome_2bit, Coordinate #https://github.com/lucapinello/bioutilities
import os

In [6]:
#load genome sequence, here we use mm10
genome=Genome_2bit('/homes10/lpinello/Haystack_dependencies/genomes/mm10.2bit') 

# you can download the mm10 genome in 2bit format from UCSC Genome Browser with
# wget http://hgdownload.soe.ucsc.edu/goldenPath/mm10/bigZips/mm10.2bit

In [8]:
#simple wrapper to perform a denovo motif analysis based on homer (http://homer.ucsd.edu/homer/motif/)
def scan_denovo_homer(target_filename,bg_filename,window_size):
    name='HOMER_DENOVO_%s_vs_%s' %(os.path.basename(target_filename),os.path.basename(bg_filename))
    target_fasta=target_filename.replace('.bed','.%dbp.fa' %window_size)
    bg_fasta=bg_filename.replace('.bed','.%dbp.fa' %window_size)
    
    target=Coordinate.bed_to_coordinates(target_filename)
    background=Coordinate.bed_to_coordinates(bg_filename)
    
    target_window=Coordinate.coordinates_of_intervals_around_center(target,window_size)
    background_window=Coordinate.coordinates_of_intervals_around_center(background,window_size)
    
    Coordinate.coordinates_to_fasta(target_window,target_fasta,genome)
    Coordinate.coordinates_to_fasta(background_window,bg_fasta,genome)
    !findMotifs.pl {target_fasta} fasta motifResults/{name} -fasta {bg_fasta} -noknown -cache 8000 


In [9]:
window_size=100 #100bp around the putative CRISPR SNPs
bg_filename='GATK.dbSNP.bed' #as background we use common SNPs recovered in any mice with GATK

In [11]:
# For target sets we use pairwise comparions from the cancer pipeline and also inverting cancer and normal i.e.:
#1) F03 VS FVB
#2) F05 VS FVB
#3) FVB VS F03
#4) FVB VS F05
for target_filename in glob.glob('*.filt.bed'):
    print target_filename
    scan_denovo_homer(target_filename,bg_filename,window_size)

treatedFVB_normalF05.filt.bed

Selected Options:
	Input file = treatedFVB_normalF05.filt.100bp.fa
	Promoter Set = fasta
	Output Directory = motifResults/HOMER_DENOVO_treatedFVB_normalF05.filt.bed_vs_GATK.dbSNP.bed
	Will use FASTA files for motif finding
		Target Sequences = treatedFVB_normalF05.filt.100bp.fa
		Background Sequences = GATK.dbSNP.100bp.fa
	Will not search for known motifs
	Using custom gene IDs for GO analysis
	Parsing FASTA format files...
	Found 300 sequences
	Found 31078 sequences

	Progress: Step4 - removing redundant promoters

	Progress: Step5 - adjusting background sequences for GC/CpG content...

	Sequences processed:
		Auto detected maximum sequence length of 101 bp
		31378 total

	Frequency Bins: 0.2 0.25 0.3 0.35 0.4 0.45 0.5 0.6 0.7 0.8
	Freq	Bin	Count
	0.2	0	326
	0.25	1	788
	0.3	2	2374
	0.35	3	4459
	0.4	4	6204
	0.45	5	6033
	0.5	6	5636
	0.6	7	4906
	0.7	8	566
	0.8	9	82
	10	10	4
	Bin	# Targets	# Background	Background Weight
	0	4	322	1.287
	1	9	779	1.197
	2	22	23

In [7]:
!findMotifs.pl


	Program will find de novo and known motifs in a gene list

		Usage:  findMotifs.pl <input list> <promoter set> <output directory> [additoinal options]

		example: findMotifs.pl genelist.txt mouse motifResults/ -len 10

		FASTA example: findMotifs.pl targets.fa fasta motifResults/ -fasta background.fa

	Available Promoter Sets: Add custom promoters sets with loadPromoters.pl

		Try typing "perl /gcdata/gcproj/Luca/HOMER/.//configureHomer.pl -list" to see available promoter sets
		Typing "perl /gcdata/gcproj/Luca/HOMER/.//configureHomer.pl -install NNN" to install promoter set NNN

	Basic options:
		-len <#>[,<#>,<#>...] (motif length, default=8,10,12) [NOTE: values greater 12 may cause the program
			to run out of memmory - in these cases decrease the number of sequences analyzed]
		-bg <background file> (ids to use as background, default: all genes)
		-start <#> (offset from TSS, default=-300) [max=based on Promoter Set]
		-end <#> (offset from TSS, default=50) [max=based on Promoter